In [1]:
import numpy as np

In [32]:
'''Define the constants'''
G = 6.6743e-11      # Gravitational Constant
Me = 5.972e24       # Mass of Earth
a = 25512e3         # Semi Major Axis of the orbit
e = 0.625           # Eccentricity of the orbit
t = 4 * 3600        # Seconds since the periapsis


'''Calculate the Gravitational parameter for Earth'''
mu_e = G * Me    # mu = GM

'''Calculate the average angular speed of an object in its orbit around another object'''
n = np.sqrt(mu_e/a**3)    # n = sqrt(mu/a^3)

'''Calculate the Mean Anomaly since the 4 hours of the periapsis'''
M = n * t    # M = nt

'''Print the value'''
print(f'The Mean Anomaly since the periapsis is {round(M, 3)} rad')

The Mean Anomaly since the periapsis is 2.231 rad


In [33]:
# Defining Functions for Kepler Equation, Derivative of Kepler Equation and Newton–Raphson
# ---------------------------------------------------------
# 1. Kepler's Equation:  M = E - e*sin(E)
# ---------------------------------------------------------
def kepler_eq(E, M, e):
    '''
    Kepler's Equation:-
    M = E - e sin(E)

    Converting it to the format of f(x) = 0
    E - e sin(E) - M = 0
    '''
    return E - e*np.sin(E) - M

In [34]:
# ---------------------------------------------------------
# 2. Derivative: d/dE [E - e*sin(E)] = 1 - e*cos(E)
# ---------------------------------------------------------
def derivative_kepler_eq(E, e):
    '''
    Derivative of Kepler's Equation for Newton–Raphson:
    M = E - e sin(E)
    dM/dE = 1 - e cos(E)
    '''
    return 1 - e * np.cos(E)

In [35]:
# ---------------------------------------------------------
# 3. Newton–Raphson Solver for Eccentric Anomaly
# ---------------------------------------------------------
def newton_raphson(func, dfunc, x0, eps=1e-10, max_iter=50):
    """
    Newton–Raphson Method

    func  - Kepler's Equation in form f(E) = 0
    dfunc - Derivative of Kepler's Equation
    x0    - Initial guess for E
    eps   - Tolerance for convergence
    """

    xn = x0               # current guess
    fn = func(xn)         # f(E)
    iteration = 0
    while abs(fn) > eps and iteration < max_iter:
        xn = xn - fn / dfunc(xn)
        fn = func(xn)
        iteration += 1
        # Safety: stop runaway iterations
        if iteration == 100:
            print("⚠️ Failed to converge after 100 iterations.")
            break

    return xn

In [36]:
# ---------------------------------------------------------
# 4. Solve Full Kepler Problem
# Input:
#   a  = semi-major axis (meters)
#   e  = eccentricity
#   mu = gravitational parameter GM
#   t  = time since periapsis (seconds)
# Output:
#   r  = orbital radius
#   v  = true anomaly
#   x,y = orbital plane coordinates
# ---------------------------------------------------------
def kepler_solver(a, e, mu, t):
    # Mean motion (average angular speed)
    n = np.sqrt(mu / a**3)

    # Mean anomaly
    M = n * t    # rad

    # Wrap M into 0 – 2π
    M = M % (2*np.pi)

    # Initial guess: E0 ≈ M for small e or pi for high e
    E0 = M if e < 0.8 else np.pi

    # Solve Kepler's equation using Newton–Raphson
    func  = lambda E: kepler_eq(E, M, e)
    dfunc = lambda E: derivative_kepler_eq(E, e)
    E = newton_raphson(func, dfunc, E0)

    # True Anomaly (ν)
    v = 2 * np.arctan2(
        np.sqrt(1 + e) * np.sin(E / 2),
        np.sqrt(1 - e) * np.cos(E / 2)
    )

    # Orbital radius
    r = a * (1 - e * np.cos(E))

    # Position in orbital plane
    x = r * np.cos(v)
    y = r * np.sin(v)

    return {
        "M": M,
        "E": E,
        "true_anomaly": v,
        "radius": r,
        "x": x,
        "y": y
    }

In [37]:



result = kepler_solver(a, e, mu, t)

print("Mean Anomaly:", result["M"])
print("Eccentric Anomaly:", result["E"])
print("True Anomaly:", result["true_anomaly"])
print("Radius:", result["radius"])
print("Position (x, y):", result["x"], result["y"])


Mean Anomaly: 2.2310458427066693
Eccentric Anomaly: 2.5694451076816747
True Anomaly: 2.8608488483501637
Radius: 38917601.692572914
Position (x, y): -37393962.70811666 10782915.861942656


In [38]:
# Initial value of E is usually used as the value of M
x0 = M

# Set the threshold value for Newton-Raphson Method
eps = 1e-6

# Find the Eccentric Anomaly for given conditions above
E = newton_raphson(
        func  = lambda E: kepler_eq(E, M, e),
        dfunc = lambda E: derivative_kepler_eq(E, e),
        x0    = x0,
        eps   = eps
    )

# Print the final value of E at given M
print(f'The Eccentric Anomaly at M = {round(M, 3)} is : {round(E, 3)} rad')


The Eccentric Anomaly at M = 2.231 is : 2.569 rad


In [39]:
# If you don't understand lambda functions, try this:

'''
Syntax of a Lambda Function:
output_of_expression = lambda argument : expression
'''

'\nSyntax of a Lambda Function:\noutput_of_expression = lambda argument : expression\n'

In [25]:
squared_val = lambda x: x**2
print(squared_val(5))   # Output: 25

25


In [40]:
'''
Equation -

f = 2 arctan( sqrt((1 + e)/(1 - e)) tan(E/2) )
'''

# Break the equation into multiple step
sqrt_term = np.sqrt((1 + e)/(1 - e))
tan_term  = np.tan(E/2)
f = 2 * np.arctan(sqrt_term * tan_term)

print(f'The True Anomaly at given postion is {round(f, 3)} rad')

The True Anomaly at given postion is 2.861 rad


In [41]:
'''
Equation:-
r = a(1 - e^2) / (1 + e cos f)
'''

r = a * (1 - e**2) / (1 + e * np.cos(f))

print(f'Distance between the Earth and the Satellite at True Anomaly of {round(f, 3)} is {round(r/10**3, 3)} km')


Distance between the Earth and the Satellite at True Anomaly of 2.861 is 38917.602 km


In [42]:
'''
Formula:-
v = sqrt(mu(2/r - 1/a))
'''

v = np.sqrt(mu_e * (2/r - 1/a))

print(
    f'The velocity of the satellite at distance of {round(r/10**3, 3)} km from the Earth '
    f'is {round(v/10**3, 3)} km/s'
)


The velocity of the satellite at distance of 38917.602 km from the Earth is 2.205 km/s


In [30]:
# Convert this true anomaly from rad into deg
2.861 * 180 / np.pi


163.92322518692853

In [44]:
np.degrees(round(f,3))


np.float64(163.92322518692853)

The True Anomaly at given postion is 2.861 rad
